# 1. Décorateur:
Les décorateurs sont des fonctions dont le rôle est de modifier le comportement par défaut d'autres fonctions ou classes.

Une fonction modifiée par un décorateur ne s'exécutera pas elle-même mais appellera le décorateur. C'est au décorateur de décider s'il veut exécuter la fonction et dans quelles conditions.

# Paramètre:

Il prend en paramètre, une fonction (celle qu'il modifie) et renvoie une fonction (celle qui sera directement affectée à notre fonction définie). 

La fonction renvoyée peut être : 
- La même fonction
- Une autre fonction qui remplace la fonction définie pour modifier son comportement - comme le décorateur est le seul à utiliser cette fonction, elle sera donc définie à l'interieur de celui-ci.

In [1]:
# Exemple simple 
def mon_decorateur(fonction):
    """Premier exemple de décorateur"""
    print(f"Notre décorateur est appelé avec en paramètre la fonction: {fonction}")
    return fonction()

@mon_decorateur 
# Précise à Python que cette fct doit être modifiée par le décorateur.
def salut():
    """Fonction modifiée par notre décorateur"""
    print("Salut !")

## Notre décorateur est appelé avec en paramètre la fonction <function salut at 0x00BA5198>


Notre décorateur est appelé avec en paramètre la fonction: <function salut at 0x0000027D97D7F160>
Salut !


In [2]:
# Exemple avec fonction renvoyée modifiée

def mon_decorateur(fonction):
    """Notre décorateur : il va afficher un message avant l'appel de la
    fonction définie"""
    
    def fonction_modifiee():
        """Fonction que l'on va renvoyer. Il s'agit en fait d'une version
        un peu modifiée de notre fonction originellement définie. On se
        contente d'afficher un avertissement avant d'exécuter notre fonction
        originellement définie"""
        
        print(f"Attention ! On appelle {fonction}")
        return fonction()
    
    # Le décorateur renvoie une fonction de substitution
    return fonction_modifiee

@mon_decorateur
def salut():
    print("Salut !")
    
salut()

Attention ! On appelle <function salut at 0x0000027D97C6C820>
Salut !


In [3]:
# Exemple avec fonction renvoyée modifiée

def mon_decorateur(fonction):
    """Notre décorateur : il va afficher un message avant l'appel de la
    fonction définie"""
    
    def fonction_modifiee():
        """Fonction que l'on va renvoyer. Il s'agit en fait d'une version
        un peu modifiée de notre fonction originellement définie. On se
        contente d'afficher un avertissement avant d'exécuter notre fonction
        originellement définie"""
        
        print(f"Attention ! On appelle {fonction}")
        return print(f"On n'a pas renvoyé la fonction")
    
    # Le décorateur renvoie une fonction de substitution
    return fonction_modifiee

@mon_decorateur
def salut():
    print("Salut !")
    
salut()

Attention ! On appelle <function salut at 0x0000027D97D7F670>
On n'a pas renvoyé la fonction


# Exemple avec paramètre

Décorateur chargé d'exécuter une fonction en contrôlant le temps qu'elle met à s'exécuter 

Il prend en paramètre un paramètre et non une fonction

Return: Décorateur (et non une fonction de substitution)

Rappel:
------
Avant | Après 
------|-------
fonction = decorateur(fonction) | fonction = decorateur(parametre)(fonction)


In [4]:
import time

# Elle comprend trois niveaux, 
# puisque nous devons influer sur le comportement de la fct
# et que notre décorateur prend des paramètres.

def controler_temps(nb_secs):
    """Contrôle le temps mis par une fonction pour s'exécuter.
    Si le temps d'exécution est supérieur à nb_secs, on affiche une alerte"""
    
    def decorateur(fonction_a_executer):
        """Notre décorateur. C'est lui qui est appelé directement LORS
        DE LA DEFINITION de notre fonction (fonction_a_executer)"""
        
        def fonction_modifiee(*parametres_non_nommes, **parametres_nommes):
            """Fonction renvoyée par notre décorateur. Elle se charge
            de calculer le temps mis par la fonction à s'exécuter"""
            
            tps_avant = time.time() # Avant d'exécuter la fonction
            valeur_renvoyee = fonction_a_executer(*parametres_non_nommes, **parametres_nommes) # On exécute la fonction
            tps_apres = time.time()
            tps_execution = tps_apres - tps_avant
            if tps_execution >= nb_secs:
                print("La fonction {0} a mis {1} pour s'exécuter".format( \
                        fonction_a_executer, tps_execution))
            return valeur_renvoyee
        return fonction_modifiee
    return decorateur

In [5]:
@controler_temps(4)
def attendre():
    input("Appuyez sur Entrée...")
attendre()

Appuyez sur Entrée...


In [6]:
# Exemple - controler les ints

def controler_types(*a_args, **a_kwargs):
    """On attend en paramètres du décorateur les types souhaités. On accepte
    une liste de paramètres indéterminés, étant donné que notre fonction
    définie pourra être appelée avec un nombre variable de paramètres et que
    chacun doit être contrôlé"""
    
    def decorateur(fonction_a_executer):
        """Notre décorateur. Il doit renvoyer fonction_modifiee"""
        def fonction_modifiee(*args, **kwargs):
            """Notre fonction modifiée. Elle se charge de contrôler
            les types qu'on lui passe en paramètres"""
            
            # La liste des paramètres attendus (a_args) doit être de même
            # Longueur que celle reçue (args)
            if len(a_args) != len(args):
                raise TypeError("le nombre d'arguments attendu n'est pas égal " \
                        "au nombre reçu")
            # On parcourt la liste des arguments reçus et non nommés
            for i, arg in enumerate(args):
                if a_args[i] is not type(args[i]):
                    raise TypeError("l'argument {0} n'est pas du type " \
                            "{1}".format(i, a_args[i]))
            
            # On parcourt à présent la liste des paramètres reçus et nommés
            for cle in kwargs:
                if cle not in a_kwargs:
                    raise TypeError("l'argument {0} n'a aucun type " \
                            "précisé".format(repr(cle)))
                if a_kwargs[cle] is not type(kwargs[cle]):
                    raise TypeError("l'argument {0} n'est pas de type" \
                            "{1}".format(repr(cle), a_kwargs[cle]))
            return fonction_a_executer(*args, **kwargs)
        return fonction_modifiee
    return decorateur

@controler_types(int, int)
def intervalle(base_inf, base_sup):
    print("Intervalle de {0} à {1}".format(base_inf, base_sup))

intervalle(1, 8)
# intervalle(5, "oups!")


Intervalle de 1 à 8


In [7]:
# Exemple - Compter d'appel

class Nbappel:
    def __init__(self, fct):
        self.appel = 0
        self.fct   = fct
        
    def __call__(self, *args, **kwargs):
        self.appel += 1
        print(f"{self.fct}: {self.appel} appels")
        return self.fct(*args, **kwargs)

@Nbappel
def f(*a,**b): 
    print(f"Fonction F: {a}, {b}")
    
@Nbappel
def g(*a): 
    print(f"Fonction G: {a}")

f(4, 5)
f(4, 5555)

g(434)

<function f at 0x0000027D97D7F310>: 1 appels
Fonction F: (4, 5), {}
<function f at 0x0000027D97D7F310>: 2 appels
Fonction F: (4, 5555), {}
<function g at 0x0000027D97DAB1F0>: 1 appels
Fonction G: (434,)


In [8]:
import time

def temps(f):
    """Decorateur qui calcule le temps d'execution d'une fonction"""
    def fonction_modifiee(*args, **kwargs):
        start_time = time.time()
        resultat = f(*args, **kwargs)
        end_time = time.time()
        print(f"Fonction {f}: {(end_time - start_time) / 60} s")
        return resultat #f(*args, **kwargs)
    return fonction_modifiee

@temps
def sum1(max_iter):
    return f" SUM: {sum([i for i in range(max_iter)])}"
sum1(30)

Fonction <function sum1 at 0x0000027D97DAB8B0>: 0.0 s


' SUM: 435'